## Get Started

In [2]:
# 设置环境变量
import os
os.environ["OPENAI_API_KEY"]= "sk-..."
os.environ["HTTPS_PROXY"]= "http://127.0.0.1:7890"

## Basic example: prompt + model + output parser

创建一个接受主体并生成笑话的链

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser
chain.invoke({"topic": "ice cream"})

"Why did the ice cream go to therapy?\n\nBecause it had too many toppings and couldn't handle the sprinkles!"

| 符号类似于 unix 管道运算符，它将不同的组件链接在一起，将一个组件的输出作为下一个组件的输入。

## 1.Prompt

prompt 是一个 BasePromptTemplate，

In [8]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [9]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [10]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

## 2.Model

In [11]:
message = model.invoke(prompt_value)
message

AIMessage(content="Why did the ice cream go to therapy?\n\nBecause it had too many toppings and couldn't cone-trol itself!")

In [13]:
from langchain.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nRobot: Why was the ice cream so sad? Because it was feeling a little rocky road!'

## Output parser

In [14]:
output_parser.invoke(message)

"Why did the ice cream go to therapy?\n\nBecause it had too many toppings and couldn't cone-trol itself!"

## Entire Pipeline

In [15]:
input = {"topic": "ice cream"}

prompt.invoke(input)

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [16]:
(prompt | model).invoke(input)

AIMessage(content="Why did the ice cream go to therapy?\n\nBecause it had too many toppings and couldn't find its scoop-identity!")

## RAG Search Example

对于我们的下一个示例，我们希望运行检索增强生成链以在回答问题时添加一些上下文。

In [ ]:
!pip install langchain docarray tiktoken

In [ ]:
# ValidationError: 2 validation errors for DocArrayDoc
# https://github.com/langchain-ai/langchain/issues/14585

!pip install --upgrade pydantic==1.10.13

In [4]:
# Requires:
# pip install langchain docarray tiktoken

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough


vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'